In [5]:
import requests
from PIL import Image, ImageDraw, ImageFont
import io
from textwrap import wrap
from typing import Tuple
image_url = "https://stan-tek.nyc3.cdn.digitaloceanspaces.com/memelibrary/78newlogo_trump.jpg"
text = "This is a test"
def _calculate_font_size(img: Image.Image, text: str, max_width_ratio: float = 0.9) -> Tuple[ImageFont.FreeTypeFont, int]:
        """
        Calculate the optimal font size for the text to fit the image width.
        
        Args:
            img: The image to add text to
            text: The text to add
            max_width_ratio: Maximum width of text relative to image width
            
        Returns:
            Tuple of (font, wrapped_text_lines)
        """
        # Start with a larger base size - 1/8 of image height instead of 1/5
        target_height = img.height // 8
        max_width = int(img.width * max_width_ratio)
        
        # Binary search for optimal font size
        min_size = 10
        max_size = target_height
        optimal_font = None
        optimal_wrapped_text = None
        
        while min_size <= max_size:
            current_size = (min_size + max_size) // 2
            try:
                font = ImageFont.truetype("arial.ttf", current_size)
            except:
                font = ImageFont.load_default(size=current_size)
                
            # Calculate wrapped text
            avg_char_width = font.getlength('x')
            chars_per_line = max(1, int(max_width / avg_char_width))
            wrapped_text = wrap(text, width=chars_per_line)
            
            # Check if text fits width and doesn't exceed height
            max_line_width = max(font.getlength(line) for line in wrapped_text)
            total_height = len(wrapped_text) * (font.size + 10)  # Add padding between lines
            
            if max_line_width <= max_width and total_height <= target_height * 2:
                optimal_font = font
                optimal_wrapped_text = wrapped_text
                min_size = current_size + 1  # Try larger size
            else:
                max_size = current_size - 1  # Try smaller size
        
        if optimal_font is None:
            return ImageFont.load_default(), wrap(text, width=30)
        
        return optimal_font, optimal_wrapped_text

# Download image
response = requests.get(image_url)
img = Image.open(io.BytesIO(response.content))

# Convert to RGBA if necessary
if img.mode != 'RGBA':
    img = img.convert('RGBA')

# Create drawing context
draw = ImageDraw.Draw(img)

# Calculate font size and wrap text
font, wrapped_text = _calculate_font_size(img, text)

# Calculate text positions
line_height = int(font.size * 1.2)  # Increased line spacing
total_height = len(wrapped_text) * line_height

# Position text near top with padding
top_padding = img.height * 0.02  # Reduced top padding to 2% of image height
y = top_padding

# Draw each line of text
for line in wrapped_text:
    # Calculate center position for this line
    text_width = draw.textlength(line, font=font)
    x = (img.width - text_width) / 2
    
    # Draw text outline
    outline_color = "black"
    outline_width = max(2, font.size // 12)  # Increased minimum outline width
    
    # Thicker outline for better visibility
    for dx in range(-outline_width, outline_width + 1):
        for dy in range(-outline_width, outline_width + 1):
            draw.text((x + dx, y + dy), line, font=font, fill=outline_color)
    
    # Draw main text
    draw.text((x, y), line, font=font, fill="white")
    
    # Move to next line
    y += line_height



In [6]:
# Display image with text
img.show()

In [4]:
font.